# 清洗数据

In [1]:
import pandas as pd
import numpy as np
import os
# path=r"D:\案件\山西吕梁网赌\洗钱卡流水"
# file_path=[path+'\\'+i for i in os.listdir(path)]
# file_path2=[i for i in file_path if '$' not in i ]
# file_path2,len(file_path2)

# df_list=[]
# for file_p in file_path2:
#     print(file_p)
#     df=pd.read_excel(file_p ,sheet_name='Sheet1',encoding='utf-8',dtype=object)
#     df['交易户名']=file_p.split('\\')[4]
#     print(len(df.columns))
#     df_list.append(df)
# df=pd.concat(df_list,axis=0)
# df.shape
# df.to_excel(path+"\\"+'洗钱卡汇总.xlsx',index=False,encoding='utf-8')

## 资金分析

In [2]:
import pymysql as ps
#链接数据库
db_host = 'localhost'
db_user = 'root'
db_pwd = '521521'
db_name = 'sxllwd'
char_type = 'utf-8'
db = ps.connect(db_host, db_user, db_pwd, db_name, charset='utf8')
couser = db.cursor()

In [3]:
# 获取交易卡号信息
sql='select * from gas_bank_records  '
couser.execute(sql)
rows = couser.fetchall()
des = couser.description
#获取字段名称
column_name= [item[0] for item in des]
column_name

df=pd.DataFrame(rows,columns=column_name)

In [4]:
df['jyje']=pd.to_numeric(df['jyje'])
df['jyye']=pd.to_numeric(df['jyye'])
df_select=df[['cxkh','jymc','jdbz','jysj','jyje','jyye','jydfmc','jydfzkh','beiz','zysm']]


In [5]:
#调集数据信息
print('调集人员主体数量:{} ,调集账号数量:{} ,交易总金额:{}交易总次数:{},进账金额:{} ,进账次数:{} , 出账金额:{}, 出账次数:{} ,进出账差额:{}'.format(len(df_select['jymc'].unique()),len(df_select['cxkh'].unique()),df_select['jyje'].sum(),len(df_select['cxkh']),df_select[df_select['jdbz']=='进'].jyje.sum(),df_select[df_select['jdbz']=='进'].shape[0],df_select[df_select['jdbz']=='出'].jyje.sum(),df_select[df_select['jdbz']=='出'].shape[0],df_select[df_select['jdbz']=='出'].jyje.sum()-df_select[df_select['jdbz']=='进'].jyje.sum()))

调集人员主体数量:70 ,调集账号数量:89 ,交易总金额:5624265047.9交易总次数:450116,进账金额:2805301583.27 ,进账次数:192767 , 出账金额:2818963464.63, 出账次数:257349 ,进出账差额:13661881.360000134


In [6]:
def jcz_je(row,str1):
    if row['jdbz']==str1:
        return row['jyje']
    else:
        return 0
def jcz_cs(row,str1):
    if row['jdbz']==str1:
        return 1
    else:
        return 0 
df_select['jzje']=df_select.apply(lambda x: jcz_je(x,'进'),axis=1)
df_select['jzcs']=df_select.apply(lambda x: jcz_cs(x,'进'),axis=1)
df_select['czje']=df_select.apply(lambda x: jcz_je(x,'出'),axis=1)
df_select['czcs']=df_select.apply(lambda x: jcz_cs(x,'出'),axis=1)

D:\develop\anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
D:\develop\anaconda\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
D:\develop\anaconda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

S

####  银行卡交易数据

In [42]:
df_cxkh=pd.pivot_table(df_select,index=['cxkh','jymc'],aggfunc={'jyje':[sum,len],'czje':sum,'czcs':sum,'jzje':sum,'jzcs':sum,'jysj':[min,max]}).reset_index()
df_cxkh.columns=[i+"_"+j for  i,j in df_cxkh.columns]
df_cxkh['jczce']=df_cxkh['czje_sum']-df_cxkh['jzje_sum']
df_cxkh=df_cxkh[['cxkh_', 'jymc_', 'jyje_sum','jyje_len', 'czje_sum','czcs_sum',   'jzje_sum', 'jzcs_sum','jczce','jysj_min','jysj_max']].sort_values('jyje_sum',ascending=False)
df_cxkh.columns=['交易卡号','交易名称','交易总金额','交易总次数','出账金额','出账次数','进账金额','进账次数','进出账差额','最早交易时间','最晚交易时间']
df_cxkh.to_excel('c:/Users/jxh/Desktop/吕梁网赌输出/附件二：调集银行卡资金进出情况.xlsx',index=False,encoding='utf-8')


#### 调集人员交易数据

In [41]:
df_jymc=pd.pivot_table(df_select,index=['jymc'],aggfunc={'cxkh':lambda x :len(x.unique()),'jyje':[sum,len],'czje':sum,'czcs':sum,'jzje':sum,'jzcs':sum}).reset_index()
df_jymc.columns=[i+'_'+j for i ,j in df_jymc.columns]
df_jymc=df_jymc[['jymc_', 'cxkh_<lambda>','jyje_sum', 'jyje_len', 'czcs_sum', 'czje_sum', 'jzje_sum','jzcs_sum']].sort_values('jyje_sum',ascending=False)
df_jymc.columns=['交易名称', '调集银行卡数量','交易总金额', '交易总次数', '出账金额', '出账次数', '进账金额', '进账次数']
df_jymc.to_excel('c:/Users/jxh/Desktop/吕梁网赌输出/附件三：调集人员资金进出情况.xlsx',index=False,encoding='utf-8')


#### 最大获利对手数据统计

In [162]:
df_get_balance=pd.pivot_table(df_select,index=['jydfzkh','jydfmc'],aggfunc={'czje':sum,'czcs':sum,'jzje':sum,'jzcs':sum,'jysj':[max,min]}).reset_index()
df_get_balance.columns=[i+'_'+j for i ,j in  df_get_balance.columns]
df_get_balance=df_get_balance[['jydfzkh_', 'jydfmc_', 'czcs_sum', 'czje_sum','jzcs_sum', 'jzje_sum', 'jysj_max', 'jysj_min']]
df_get_balance['jczce']=df_get_balance['czje_sum']-df_get_balance['jzje_sum']
df_get_balance=df_get_balance.sort_values('jczce',ascending=False)
df_get_balance.columns=['对手卡号','对手名称','流入笔数','流入金额','流出笔数','流出金额','最晚一次交易时间','最早一次交易时间','进出帐差额']
df_get_balance.to_excel('c:/Users/jxh/Desktop/吕梁网赌输出/附件五：最大获利对手.xlsx',index=False,encoding='utf-8')

#### 密切联系对手账号分析

In [81]:
df_get_jydf=pd.pivot_table(df_select,index=['jydfzkh','jydfmc'],aggfunc={'jymc': lambda x:len(x.unique())}).reset_index()
df_get_jydf=df_get_jydf.sort_values('jymc',ascending=False)
df_get_jydf.columns=['交易对手卡号','交易对手名称','关联调集人员']
df_get_jydf.to_excel('c:/Users/jxh/Desktop/吕梁网赌输出/附件四：密切联系对手账号分析.xlsx',index=False,encoding='utf-8')

#### 资金净流向问题

In [95]:
df_tongji=pd.pivot_table(df_select,index=['cxkh','jymc','jydfzkh','jydfmc'],aggfunc={'jyje':[sum,len],'czje':sum,'czcs':sum,'jzje':sum,'jzcs':sum,'jysj':[max,min]}).reset_index()
df_tongji.columns=[i+'_'+j for i,j in df_tongji.columns]
df_tongji=df_tongji[['cxkh_', 'jymc_', 'jydfzkh_', 'jydfmc_', 'jyje_sum', 'jyje_len','czje_sum','czcs_sum', 'jzje_sum', 'jzcs_sum',  'jysj_max', 'jysj_min']].sort_values('jyje_sum',ascending=False)
df_tongji['jczce']=df_tongji['czje_sum']-df_tongji['jzje_sum']

df_tongji.columns=['交易卡号', '交易名称', '交易对方账卡号', '交易对方名称', '交易总金额', '交易总次数','出账金额', '出账次数', '进账金额', '进账次数', '最晚一次交易时间', '最早一次交易时间','进出账差额']
df_tongji.to_excel('c:/Users/jxh/Desktop/吕梁网赌输出/附件六：整体交易数据统计汇总.xlsx',index=False,encoding='utf-8')

In [100]:
df_tongji.columns=['cxkh_', 'jymc_', 'jydfzkh_', 'jydfmc_', 'jyje_sum', 'jyje_len','czje_sum','czcs_sum', 'jzje_sum', 'jzcs_sum',  'jysj_max', 'jysj_min','jczce']
ydkh=set(df_select.cxkh)
df_tongji_100w=df_tongji[(df_tongji['jyje_sum']>1000000)  &(df_tongji['jczce']>0)]
df_tongji_100w
#判断第二层是否调单
df_tongji_100w['y_n_dd']=df_tongji_100w['jydfzkh_'].apply(lambda x: 'Y' if x in ydkh else 'N')



## 调单梳理，根据最大获利对手

In [163]:
ydkh=set(df_select.cxkh)

df_get_balance.columns=['jydfzkh_', 'jydfmc_', 'czcs_sum', 'czje_sum','jzcs_sum', 'jzje_sum', 'jysj_max', 'jysj_min','jczce']
df_get_balance['y_n_dd']=df_get_balance['jydfzkh_'].apply(lambda x: 'Y' if x in ydkh else 'N')


In [183]:
df_get_balance_100w_wei=df_get_balance[(df_get_balance['jczce']>1000000) & (df_get_balance['y_n_dd']=='N')]
df_get_balance_100w_yi=df_get_balance[(df_get_balance['jczce']>1000000) & (df_get_balance['y_n_dd']=='Y')]

In [185]:
df_two=pd.merge(df_get_balance_100w_yi,df_tongji,left_on='jydfzkh_',right_on='cxkh_',how='left')

In [192]:
writer=pd.ExcelWriter("c:/Users/jxh/Desktop/吕梁网赌输出/附件八调单数据.xlsx")
df_get_balance_100w_wei.to_excel(writer,sheet_name='二层未调单数据梳理',index=False,encoding='utf-8')
df_two.to_excel(writer,sheet_name='二层调单数据梳理',index=False,encoding='utf-8')
writer.save()
writer.close()



In [196]:
df_get_balance[(df_get_balance['jczce']>1000000) & (df_get_balance['y_n_dd']=='N')].shape

(156, 10)

In [145]:
writer=pd.ExcelWriter("'c:/Users/jxh/Desktop/吕梁网赌输出/附件八：调单数据.xlsx",index=False,encoding='utf-8')
df_tongji_100w.to_excel(writer,sheet_name='二层未调单数据梳理')



205